In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from torchsummary import summary
import numpy as np

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/car_evaluation.csv')

In [4]:
data

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...
1722,low,low,5more,more,med,med,good
1723,low,low,5more,more,med,high,vgood
1724,low,low,5more,more,big,low,unacc
1725,low,low,5more,more,big,med,good


In [5]:
data['unacc'].value_counts()

,count
unacc,
unacc,1209
acc,384
good,69
vgood,65


In [6]:
columns = ['vhigh', 'vhigh.1', '2', '2.1', 'small', 'low', 'unacc']

In [7]:
label_encoders = {}
for column in columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

In [8]:
data

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,3,3,0,0,2,2,2
1,3,3,0,0,2,0,2
2,3,3,0,0,1,1,2
3,3,3,0,0,1,2,2
4,3,3,0,0,1,0,2
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,3
1724,1,1,3,2,0,1,2
1725,1,1,3,2,0,2,1


In [9]:
# 데이터와 타겟 분리
X = data.drop('unacc', axis=1).values
y = data['unacc'].values
Y = y

In [63]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [64]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [65]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [66]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1381, 6), (346, 6), (1381,), (346,))

In [67]:
class CarEvaluationDense(nn.Module):
    def __init__(self):
        super(CarEvaluationDense, self).__init__()
        self.fc1 = nn.Linear(6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = CarEvaluationDense()

In [68]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [69]:
# 학습 루프
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(), y_batch.to()
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.8712
Epoch 2, Loss: 0.6673
Epoch 3, Loss: 0.6372
Epoch 4, Loss: 0.5919
Epoch 5, Loss: 0.5555
Epoch 6, Loss: 0.5412
Epoch 7, Loss: 0.5032
Epoch 8, Loss: 0.4643
Epoch 9, Loss: 0.4532
Epoch 10, Loss: 0.4457
Epoch 11, Loss: 0.4237
Epoch 12, Loss: 0.4108
Epoch 13, Loss: 0.4056
Epoch 14, Loss: 0.3694
Epoch 15, Loss: 0.3471
Epoch 16, Loss: 0.3203
Epoch 17, Loss: 0.3135
Epoch 18, Loss: 0.2937
Epoch 19, Loss: 0.2752
Epoch 20, Loss: 0.2745
Epoch 21, Loss: 0.2561
Epoch 22, Loss: 0.2329
Epoch 23, Loss: 0.2221
Epoch 24, Loss: 0.2295
Epoch 25, Loss: 0.2289
Epoch 26, Loss: 0.2127
Epoch 27, Loss: 0.1873
Epoch 28, Loss: 0.1857
Epoch 29, Loss: 0.1633
Epoch 30, Loss: 0.1753
Epoch 31, Loss: 0.1791
Epoch 32, Loss: 0.1461
Epoch 33, Loss: 0.1530
Epoch 34, Loss: 0.1535
Epoch 35, Loss: 0.1245
Epoch 36, Loss: 0.1219
Epoch 37, Loss: 0.1118
Epoch 38, Loss: 0.1067
Epoch 39, Loss: 0.1056
Epoch 40, Loss: 0.1011
Epoch 41, Loss: 0.0933
Epoch 42, Loss: 0.0956
Epoch 43, Loss: 0.0858
Epoch 44, Loss: 0.08

In [70]:
# 평가
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to()
        outputs = model(X_batch).cpu().numpy()
        preds.extend(outputs)
        actuals.extend(y_batch.numpy())

mse = mean_squared_error(actuals, preds)
print(f"Test MSE: {mse:.4f}")

Test MSE: 0.0706
